In [ ]:
rain_folder = r"Data\Climate Data\3. Tasmania_Monthly\Rainfall_tas_monthly"      # folder with monthly rain .nc
pet_folder  = r"Data\Climate Data\3. Tasmania_Monthly\ET_tas_monthly"      # folder with monthly FAO56 ET .nc
out_folder  = r"Data\Climate Data\3. Tasmania_Monthly\SPEI_tas_monthly"           # where to save SPEI files

In [ ]:
import os
import xarray as xr
import numpy as np
from scipy.stats import fisk, norm  # pip install scipy

# -------------------------------------------------------
# 1. USER SETTINGS – EDIT THESE
# -------------------------------------------------------

rain_dir = r"Data\Climate Data\3. Tasmania_Monthly\Rainfall_tas_monthly"      # folder with ALL monthly rain .nc
pet_dir  = r"Data\Climate Data\3. Tasmania_Monthly\ET_tas_monthly"       # folder with ALL monthly ET .nc
out_dir  = r"Data\Climate Data\3. Tasmania_Monthly\SPEI_tas_monthl"       # folder to save SPEI yearly files

os.makedirs(out_dir, exist_ok=True)

# variable names inside the .nc files
RAIN_VAR = "monthly_rain"      # e.g. "monthly_rain", "precip"
PET_VAR  = "et_short_crop"       # e.g. "ET_shortcrop", "pet"

SPEI_VAR_NAME = "spei"
TIMESCALE = 1          # 1-month SPEI

# -------------------------------------------------------
# 2. OPEN ALL RAIN & PET FILES AS ONE TIME SERIES
# -------------------------------------------------------

def list_nc(folder):
    return sorted(
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if f.endswith(".nc")
    )

rain_files = list_nc(rain_dir)
pet_files  = list_nc(pet_dir)

if not rain_files:
    raise RuntimeError(f"No .nc files found in {rain_dir}")
if not pet_files:
    raise RuntimeError(f"No .nc files found in {pet_dir}")

print(f"Found {len(rain_files)} rain files and {len(pet_files)} PET files.")

rain_ds = xr.open_mfdataset(
    rain_files,
    combine="by_coords",
    parallel=True,
)
pet_ds = xr.open_mfdataset(
    pet_files,
    combine="by_coords",
    parallel=True,
)

rain = rain_ds[RAIN_VAR]
pet  = pet_ds[PET_VAR]

# make sure coords match
if not rain.time.identical(pet.time):
    raise ValueError("Rain and PET time coordinates do not match.")
for coord in ["lat", "lon"]:
    if coord in rain.coords and coord in pet.coords:
        if not rain[coord].identical(pet[coord]):
            raise ValueError(f"Rain and PET {coord} coordinates do not match.")

print("Rain & PET datasets aligned in time and space.")

# -------------------------------------------------------
# 3. WATER BALANCE (P - PET)
# -------------------------------------------------------

wb = rain - pet
wb = wb.rename("water_balance")
wb.attrs["long_name"] = "Precipitation minus evapotranspiration"
wb.attrs["units"] = rain.attrs.get("units", "")

# For multi-month SPEI, do rolling sum
if TIMESCALE > 1:
    wb = wb.rolling(time=TIMESCALE, min_periods=TIMESCALE).sum()

# IMPORTANT: fix dask core-dimension issue (time in a single chunk)
wb = wb.chunk({"time": -1, "lat": 50, "lon": 50})  # adjust 50 if needed

# -------------------------------------------------------
# 4. 1D SPEI FUNCTION
# -------------------------------------------------------

def spei_1d(x_1d):
    """
    Compute SPEI from a 1D water-balance series using
    log-logistic (fisk) + inverse normal.
    """
    x = np.asarray(x_1d, dtype=float)
    out = np.full_like(x, np.nan, dtype=float)

    mask = np.isfinite(x)
    if mask.sum() < 10:
        return out

    try:
        c, loc, scale = fisk.fit(x[mask])
    except Exception:
        return out

    F = fisk.cdf(x[mask], c, loc=loc, scale=scale)
    eps = 1e-6
    F = np.clip(F, eps, 1 - eps)

    z = norm.ppf(F)
    out[mask] = z
    return out

# -------------------------------------------------------
# 5. APPLY SPEI OVER ALL GRID CELLS
# -------------------------------------------------------

spei_da = xr.apply_ufunc(
    spei_1d,
    wb,
    input_core_dims=[["time"]],
    output_core_dims=[["time"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[float],
)

spei_da = spei_da.rename(SPEI_VAR_NAME)
spei_da.attrs["long_name"] = f"SPEI ({TIMESCALE}-month)"
spei_da.attrs["units"] = "1"

spei_ds = spei_da.to_dataset()

# -------------------------------------------------------
# 6. SAVE ONE FILE PER YEAR (EACH CONTAINING 12 MONTHS)
# -------------------------------------------------------

years = np.unique(spei_ds["time"].dt.year.values)

for y in years:
    year_str = str(y)
    sub = spei_ds.sel(time=spei_ds["time"].dt.year == y)

    if sub.time.size == 0:
        continue

    out_name = f"SPEI_{TIMESCALE}m_{year_str}.nc"
    out_path = os.path.join(out_dir, out_name)
    print(f"Saving {out_name} with {sub.time.size} months.")
    sub.to_netcdf(out_path)

print("All yearly SPEI files saved.")


Loading data from: Data\Climate Data. Tasmania_Monthly\Rainfall_tas_monthly
Error: No NetCDF files found in Data\Climate Data. Tasmania_Monthly\Rainfall_tas_monthly
Please ensure the RAINFALL_DIR and EVAPOTRANSPIRATION_DIR are correct and contain .nc files.


<>:11: SyntaxWarning: invalid escape sequence '\C'
<>:12: SyntaxWarning: invalid escape sequence '\C'
<>:13: SyntaxWarning: invalid escape sequence '\C'
<>:11: SyntaxWarning: invalid escape sequence '\C'
<>:12: SyntaxWarning: invalid escape sequence '\C'
<>:13: SyntaxWarning: invalid escape sequence '\C'
C:\Users\u1171032\AppData\Local\Temp\ipykernel_29500\2744082860.py:11: SyntaxWarning: invalid escape sequence '\C'
  RAINFALL_DIR = "Data\Climate Data\3. Tasmania_Monthly\Rainfall_tas_monthly"
C:\Users\u1171032\AppData\Local\Temp\ipykernel_29500\2744082860.py:12: SyntaxWarning: invalid escape sequence '\C'
  EVAPOTRANSPIRATION_DIR = "Data\Climate Data\3. Tasmania_Monthly\ET_tas_monthly"
C:\Users\u1171032\AppData\Local\Temp\ipykernel_29500\2744082860.py:13: SyntaxWarning: invalid escape sequence '\C'
  OUTPUT_DIR = "Data\Climate Data\3. Tasmania_Monthly\SPEI_tas_monthlyyyy" # New directory for annual files
